# Lab Notebook: Simple Usage Example

This notebook demonstrates how to use labnotebook with randomly generated experimental data.

In [1]:
import labnotebook
import numpy as np

## Setup

We first set up our model; it will generate random numbers between 0 and 100 for train accuracy, validation accuracy, and train loss.

Our model will have only one "hyperparameter": the volatility of the random number generation; this would typically be a dictionnary containing *all* the hyperparameters of your model.

Also, we'll break out the generation of each random number into a `for` loop, which obviously doesn't make any sense. Every step of this loop would translate into one step of your optimisation algorithm.

In [2]:
model_desc = {'sigma': .1}

The first step is to initialize the package by providing it the address of the database you want to use.

It will create three tables: `experiments`, `steps`, and `model_params`.

- `experiments` is used to store a list of experiments, along with their hyperparameters and final results.

- `steps` is used to store the intermediary results for each step of each experiment. This is what you would want to plot if you're monitoring your experiments.

- Finally, `model_params` is used to store your model parameters; what you would use to save the weights of your neural network for later inference. This can get pretty big so it's recommended not to save all the parameters at every step.

In [3]:
db_url = 'postgres://<username>:<passsword>@localhost/<database_name>'

In [4]:
experiments, steps, model_params = labnotebook.initialize(db_url)

/home/henripal/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Running an Experiment

Here, our virtual experiment will generate random numbers for 10 steps.

There are only three extra lines of code added to permanently record this experiment in your database and plot it using the web app: `start_experiment`, `step_experiment` and `stop_experiment`.

In [5]:
# we start the experiment and output it to an 'experiment' variable
# we can then pass this experiment to step_experiment and end_experiment
experiment = labnotebook.start_experiment(model_desc = model_desc)


for experiment_step in range(10):
    
    # randomly generated validation, training accuracies and losses:
    valacc = np.random.normal(0, model_desc['sigma'])
    trainacc = np.random.normal(0, model_desc['sigma'])
    trainloss = np.random.normal(0, model_desc['sigma'])
    
    # example 'custom field' that you can add to your tracking
    other_variable = experiment_step * 2 - valacc
    
    # we pass all our indicators to step_experiment
    labnotebook.step_experiment(experiment,
                                timestep=experiment_step,
                                trainloss=trainloss,
                                valacc=valacc,
                                trainacc=trainacc,
                                custom_fields={'goofy_variable': other_variable})
    
    
# we close the experiment and pass all final indicators:
labnotebook.end_experiment(experiment,
                            final_trainloss=trainloss,
                            final_valacc=valacc,
                            final_trainacc=trainacc)
    

Run 6 on GPU 0 at 2018-03-18 14:53:17.418460

## Accessing our experiments
### Through the web app

Two steps are needed:
- Launch the backend flask API by running from the command line:
```
start_backend <database_url>
```

- Navigate to the `labnotebook/frontend` directory and serve its contents, for example by running `python -m http.server`
You should see something like this after selecting experiments from the left menu:

![](./img/labnotebook.png)

You can change what you see, turn live updating on or off, etc... from the `options` menu. 

## Through sqlalchemy ORM commands

You essentially have access to *all* your data in a relational database, and can query it in sophisticated ways that are beyond the scope of this notebook. 

I recommend looking through [sqlalchemy's documentation](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html#querying) , but here are some simple example queries.


In [6]:
# list all experiments and print some of their properties:

experiment_list = labnotebook.session.query(experiments).all()

for experiment in experiment_list: 
    print("run id: ", experiment.run_id, end='\t')
    print("model_desc: ", experiment.model_desc)

run id:  1	model_desc:  {'sigma': 0.1}
run id:  2	model_desc:  {'sigma': 0.1}
run id:  3	model_desc:  {'sigma': 0.1}
run id:  4	model_desc:  {'sigma': 0.1}
run id:  5	model_desc:  {'sigma': 0.1}
run id:  6	model_desc:  {'sigma': 0.1}


In [7]:
# list all steps of experiment #1 and print train accuracies:

step_list = labnotebook.session.query(steps).filter(steps.run_id == 4).all()

for step in step_list:
    print("training accuracy: ", step.trainacc)

training accuracy:  0.0881643584718841
training accuracy:  0.114199389001045
training accuracy:  -0.20606842946817
training accuracy:  0.00735350586226073
training accuracy:  -0.149939680518542
training accuracy:  -0.146279798077668
training accuracy:  0.0126899091536999
training accuracy:  -0.0281140716874202
training accuracy:  0.0586363642532856
training accuracy:  0.00123095750017543


In [8]:
# list all experiments where sigma is greater than 0:
import sqlalchemy

experiment_list_highsigma = labnotebook.session.query(
    experiments).filter(experiments.model_desc['sigma'].astext.cast(sqlalchemy.types.Float) > 0).all()

for experiment in experiment_list_highsigma:
    print(experiment)

Run 1 on GPU 0 at 2018-03-18 14:11:50.702179
Run 2 on GPU 0 at 2018-03-18 14:11:50.702179
Run 3 on GPU 0 at 2018-03-18 14:11:50.702179
Run 4 on GPU 0 at 2018-03-18 14:11:50.702179
Run 5 on GPU 0 at 2018-03-18 14:52:56.322151
Run 6 on GPU 0 at 2018-03-18 14:53:17.418460


Note in this last example that we're filtering with respect to items inside a dictionary; they are passed as text so we have to cast them to Float to run comparisons.